In [10]:
import pandas as pd
import numpy as np
import glob

In [11]:
df = pd.read_csv('estat_educ_uoe_enrt01.csv')
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
df.head()

,freq,unit,worktime,sector,sex,isced11,geo,2005,2010,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,A,NR,FT,PRIV,F,ED5,AL,:,:,:,:,:,:,:,:,:,:,:,412,481
1,A,NR,FT,PRIV,F,ED5,AT,:,:,:,7576,7662,8161,7933,8157,8325,8378,8307,9088,9416
2,A,NR,FT,PRIV,F,ED5,BA,:,:,:,:,:,:,:,:,: z,: z,: z,: z,0
3,A,NR,FT,PRIV,F,ED5,BE,:,:,:,4441,4766,4741,4721,4454,4162,4072,4893 bd,6034,5580
4,A,NR,FT,PRIV,F,ED5,BG,:,:,:,: z,: z,: z,: z,: z,: z,: z,: z,: z,: z


In [12]:
path = '../ESTAT_CODELISTS/*.tsv'
files = glob.glob(path)

dataframes = {file.split('/')[-1].replace('.tsv', ''): pd.read_csv(
    file, sep='\t') for file in files}

In [13]:
for col in ["freq", "unit", "worktime", "sector", "sex", "isced11", "geo"]:
    curr_codelist = dataframes[f"ESTAT_{col.upper()}"]
    df[col] = df[col].map(curr_codelist.set_index('CODE')['Label'])

df.rename(columns={'geo': 'country'}, inplace=True)

df.head()

,freq,unit,worktime,sector,sex,isced11,country,2005,2010,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Annual,Number,Full-time,Private institutions,Females,Short-cycle tertiary education,Albania,:,:,:,:,:,:,:,:,:,:,:,412,481
1,Annual,Number,Full-time,Private institutions,Females,Short-cycle tertiary education,Austria,:,:,:,7576,7662,8161,7933,8157,8325,8378,8307,9088,9416
2,Annual,Number,Full-time,Private institutions,Females,Short-cycle tertiary education,Bosnia and Herzegovina,:,:,:,:,:,:,:,:,: z,: z,: z,: z,0
3,Annual,Number,Full-time,Private institutions,Females,Short-cycle tertiary education,Belgium,:,:,:,4441,4766,4741,4721,4454,4162,4072,4893 bd,6034,5580
4,Annual,Number,Full-time,Private institutions,Females,Short-cycle tertiary education,Bulgaria,:,:,:,: z,: z,: z,: z,: z,: z,: z,: z,: z,: z


In [14]:
df.replace(": z", np.nan, inplace=True)
df.replace(":", np.nan, inplace=True)

In [15]:
year_cols = [col for col in df.columns if col.isdigit()]

for col in year_cols:
    df[col] = df[col].astype(str).str.extract(r'(\d+)').astype(float)

In [16]:
if df["freq"].nunique() == 1:
    df.drop(columns=["freq"], inplace=True)
if df["unit"].nunique() == 1:
    df.drop(columns=["unit"], inplace=True)

df.rename(columns={'isced11': 'level'}, inplace=True)

In [17]:
df.head()

,worktime,sector,sex,level,country,2005,2010,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Full-time,Private institutions,Females,Short-cycle tertiary education,Albania,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,412.0,481.0
1,Full-time,Private institutions,Females,Short-cycle tertiary education,Austria,NaN,NaN,NaN,7576.0,7662.0,8161.0,7933.0,8157.0,8325.0,8378.0,8307.0,9088.0,9416.0
2,Full-time,Private institutions,Females,Short-cycle tertiary education,Bosnia and Herzegovina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,Full-time,Private institutions,Females,Short-cycle tertiary education,Belgium,NaN,NaN,NaN,4441.0,4766.0,4741.0,4721.0,4454.0,4162.0,4072.0,4893.0,6034.0,5580.0
4,Full-time,Private institutions,Females,Short-cycle tertiary education,Bulgaria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
df.to_csv("./clean/estat_educ_uoe_enrt01.csv", index=False)